In [2]:
!pip install numpy pandas matplotlib seaborn scikit-learn xgboost lightgbm catboost optuna shap eli5 tensorflow keras torch torchvision

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached matplotlib-3.8.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.4.1.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached lightgbm-4.3.0-py3-none-manylinux_2_28_x86_64.whl.metadata (19 kB)
  Using cached catboost-1.2.3-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached optuna-3.5.0-py3-none-any.whl.metadata (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 2.3 MB/s eta 0:00:002.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  I

In [3]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/home/akz/programming/biotech_dataset/Train2D.csv')
df.head()

,SMILES,Cls,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C(=...,0,NaN,NaN,1,10,126.291271,2.416707,4.779198,126.291271,...,11.160285,157.462679,1463.720435,7.318602,67876,151,492,548,40.194444,23.333333
1,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H...,0,NaN,NaN,1,10,122.306081,2.416770,4.780587,122.306081,...,11.127219,155.307010,1441.772471,7.067512,63926,148,478,531,41.194444,22.944444
2,C1=CC=C(C(=C1)C(=O)C2=CN=C(S2)NC3=CC=C(C=C3)N)Cl,0,NaN,NaN,0,0,28.571924,2.388244,4.711441,28.571924,...,9.874522,69.492153,329.038961,9.677616,1165,31,114,132,6.777778,4.833333
3,C1CCC(=O)C(C1)C2CC(=O)N(C2=O)C3=CC=C(C=C3)Br,0,NaN,NaN,0,0,27.097139,2.488091,4.835581,27.097139,...,10.022470,69.396660,349.031356,9.433280,902,34,112,134,7.138889,4.583333
4,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H...,0,NaN,NaN,1,10,132.321058,2.416813,4.810016,132.321058,...,11.267217,163.855774,1557.759855,7.382748,77098,162,524,585,43.618056,24.527778
